<a href="https://colab.research.google.com/github/awchisholm/snippets/blob/main/automated_unit_testing_with_pytest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q ipytest # not needed anymore

In [ ]:
import pytest
import ipytest
ipytest.autoconfig()

In [ ]:
# Some code we want to test later
import sqlite3
import os
import os.path

# Define the DDL SQL 
sql = """
CREATE TABLE "customer" (
	"Id"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	"Name"	TEXT,
	"Height"	REAL
);
CREATE TABLE "booking" (
	"Id"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	"Details"	TEXT,
	"CustomerId"	INTEGER,
	CONSTRAINT "customerid_fk" FOREIGN KEY("CustomerId") REFERENCES "customer"("Id")
);
"""

database_file = 'customer.db'
# Delete the database
if os.path.exists(database_file):
  os.remove(database_file)

# Connect to the database
conn = sqlite3.connect(database_file)
# Get a cursor pointing to the database
cursor = conn.cursor()
# Create the tables
cursor.executescript(sql)
# Commit to save everything
conn.commit()
# Close the connection to the database
conn.close()

def query_database(database, query):
    conn = sqlite3.connect(database)
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    cur.close()
    return rows

def execute_sql(database, sql_statement):
    conn = sqlite3.connect(database)
    #conn.execute("PRAGMA foreign_keys = 1")
    cur = conn.cursor()
    cur.execute(sql_statement)
    conn.commit()
    return cur.lastrowid

insert = "insert into customer (name, height) values ('Andrew Chisholm', 1.73)"
execute_sql(database_file, insert)
result = query_database(database_file, "select * from customer")

In [ ]:
%%ipytest
# This causes ipytest to look for all functions beginning with test_
# and it then runs them to produce a nice looking report
# The %%ipytest directive above is used in Colab - not needed in Visual Studio

#1,2,4,5 will pass: 3 will fail

def test_database_exists():
    assert os.path.exists(database_file) == True

def test_database_contains_one_row():
    assert len(query_database(database_file, 'select * from customer')) == 1

def test_database_contains_expected_row():
    fetched = query_database(database_file, 'select Name, Height from customer')
    expected = [('Andrew Chisholm', 1.72)]
    # This will fail
    assert fetched == expected

def test_row_deleted_from_database():
    result = execute_sql(database_file, "delete from customer where name = 'Andrew Chisholm'")
    fetched = query_database(database_file, 'select * from customer')
    expected = []
    assert fetched == expected

def test_row_added_to_database():
    insert = "insert into customer (name, height) values ('Andrew Chisholm', 1.73)"
    execute_sql(database_file, insert)
    fetched = query_database(database_file, 'select Name, Height from customer')
    expected = [('Andrew Chisholm', 1.73)]
    assert fetched == expected

..F..                                                                                        [100%]
============================================= FAILURES =============================================
_______________________________ test_database_contains_expected_row ________________________________

    def test_database_contains_expected_row():
        fetched = query_database(database_file, 'select Name, Height from customer')
        expected = [('Andrew Chisholm', 1.72)]
        # This will fail
>       assert fetched == expected
E       AssertionError: assert [('Andrew Chisholm', 1.73)] == [('Andrew Chisholm', 1.72)]
E         At index 0 diff: ('Andrew Chisholm', 1.73) != ('Andrew Chisholm', 1.72)
E         Full diff:
E         - [('Andrew Chisholm', 1.72)]
E         ?                         ^
E         + [('Andrew Chisholm', 1.73)]
E         ?                         ^

<ipython-input-11-766547ab3c41>:15: AssertionError
===================================== short test summary 